# Hướng dẫn Deploy FOODFAST lên AWS EC2

Tài liệu này mô tả cách deploy ứng dụng FOODFAST lên một instance AWS EC2 bằng GitHub Actions (workflow `Auto Deploy Foodfast to EC2`) được tìm thấy trong `.github/workflows/deploy-production.yml`.

Mục lục
- Yêu cầu trước
- Các secrets/biến cần thiết
- Thiết lập EC2 (tạo instance, security group, SSH keys)
- Cài Docker trên EC2
- Cách workflow GitHub Actions hoạt động
- Cách chạy thủ công (nếu cần)
- Kiểm tra, troubleshooting

---

## 1. Yêu cầu trước
- Tài khoản AWS với quyền tạo EC2 và thay đổi Security Group.
- Một EC2 instance (khuyến nghị Ubuntu LTS) có thể truy cập SSH từ runner GitHub Actions.
- Docker được cài trên EC2 (workflow sử dụng `docker` để pull/run container).
- 1 GitHub repository có workflow `.github/workflows/deploy-production.yml` (đã có sẵn).
- Các Docker image được push lên `ghcr.io/ductoanoxo/...` (hoặc registry tương ứng).

## 2. Secrets & biến cần cấu hình trên GitHub (Repository -> Settings -> Secrets & variables -> Actions)
Workflow hiện tại dùng các secrets sau (bắt buộc):
- `PROD_SSH_KEY` — private SSH key (dạng PEM) để GitHub Actions kết nối SSH tới EC2.
  - Key này phải tương ứng với public key được thêm vào `~/.ssh/authorized_keys` của user trên EC2.
  - Nếu private key có passphrase, workflow với `webfactory/ssh-agent` có thể xử lý, nhưng khuyến nghị dùng key không có passphrase cho automation.
- `PROD_SERVER_HOST` — địa chỉ IP công khai hoặc DNS của EC2 (vd: `203.0.113.12`).
- `PROD_SERVER_USER` — tên user trên EC2 (ví dụ `ubuntu` hoặc `ec2-user`).
- `GHCR_TOKEN` — GitHub Personal Access Token để login vào GHCR (nếu images private). Token nên có scope `read:packages` (và `repo` nếu cần truy cập package private).
- `MONGO_URI` — connection string tới MongoDB (nếu server cần). Có thể là Atlas hoặc một DB reachable từ EC2.

Ngoài ra workflow dùng một số biến môi trường trong file yml (có thể override nếu cần):
- `REGISTRY` (mặc định `ghcr.io`)
- `OWNER` (mặc định `ductoanoxo`)
- `SERVER_TAG`, `RESTAURANT_TAG`, `CLIENT_TAG`, `ADMIN_TAG`, `DRONE_TAG` — tags dùng để pull image.

## 3. Thiết lập EC2 (tạo instance & security)
1. Tạo EC2 instance (Ubuntu 20.04/22.04) với public IP.
2. Security Group: mở các port cần thiết:
   - 22 (SSH) — chỉ cho phép IP của GitHub Actions runners hoặc địa chỉ IP bạn cần (hoặc 0.0.0.0/0 nếu tạm thời, không khuyến nghị).
   - 5000 — backend (server) đang chạy container map tới port 5000.
   - 80/3000/3001/3002 — tùy vào mapping public của client/admin/restaurant; theo workflow client được map tới 3000:80, admin 3001:80, restaurant 3002:80.
   - Nếu dùng port khác, update security group tương ứng.
3. Tạo SSH keypair (hoặc dùng existing). Thêm public key vào `~/.ssh/authorized_keys` cho `PROD_SERVER_USER`.

## 4. Cài Docker trên EC2
SSH vào EC2 rồi chạy (ví dụ Ubuntu):

```bash
sudo apt update && sudo apt install -y docker.io
sudo systemctl enable --now docker
# (tuỳ chọn) thêm user vào group docker để chạy docker mà không cần sudo
sudo usermod -aG docker ${USER}
# relogin nếu muốn dùng docker không cần sudo
```

Workflow sử dụng `sudo docker` để tránh vấn đề quyền. Nếu muốn, bạn có thể cấu hình để không cần sudo.

## 5. Cách workflow GitHub Actions trong `deploy-production.yml` hoạt động
Tóm tắt các bước chính trong workflow:

1. Kích hoạt: workflow chạy khi push lên nhánh `main`, `kiet`, `DUCTOAN` hoặc khi trigger thủ công (`workflow_dispatch`).
2. Checkout code.
3. Thiết lập SSH agent trên runner và nạp `PROD_SSH_KEY` (`webfactory/ssh-agent`).
4. Thêm host EC2 vào `~/.ssh/known_hosts` bằng `ssh-keyscan` (dùng `PROD_SERVER_HOST`).
5. Tạo script `remote-deploy.sh` chứa logic deploy (được `scp` lên EC2 vào `/tmp/remote-deploy.sh`).
6. Chạy script trên EC2 qua SSH. Script:
   - Login GHCR với `GHCR_TOKEN` (echo token | docker login ghcr.io -u OWNER --password-stdin).
   - Tạo docker network `foodfast-net` nếu chưa có.
   - Pull image `ghcr.io/OWNER/foodfast-server:SERVER_TAG`, kill container cũ nếu có, chạy container mới map port 5000.
   - Tương tự với `foodfast-client`, `foodfast-admin`, `foodfast-restaurant`, `foodfast-drone` nếu tag khác `buildcache` (nếu tag = `buildcache` sẽ skip).
   - Các container chạy với `--restart unless-stopped`.
7. Sau đó runner thực hiện một healthcheck tới `http://PROD_SERVER_HOST:5000/health` (hoặc `/api/health` hoặc root) trong vòng 6 lần kiểm tra, mỗi lần cách nhau 5s; nếu không thành công workflow fail.

Thông tin quan trọng bạn nên biết:
- Nếu images của bạn private trên GHCR thì `GHCR_TOKEN` phải có quyền đọc package.
- Các tag mặc định trong workflow là `buildcache` hoặc `sha-cfccd2f` — bạn có thể override bằng cách chỉnh `env` trong workflow hoặc đặt secrets/inputs cho `workflow_dispatch`.

## 6. Chạy thủ công (local) — cách test script deploy
Bạn có thể chạy tương tự thủ công bằng cách copy script `remote-deploy.sh` lên EC2 và chạy:

1. Tại máy local (hoặc runner):
```bash
scp remote-deploy.sh ubuntu@YOUR_EC2_IP:/tmp/remote-deploy.sh
ssh ubuntu@YOUR_EC2_IP 'chmod +x /tmp/remote-deploy.sh && GHCR_TOKEN=your_token MONGO_URI="your_mongo_uri" REGISTRY=ghcr.io OWNER=ductoanoxo SERVER_TAG=sha-cfccd2f /tmp/remote-deploy.sh'
```
2. Hoặc inspect script rồi chạy từng lệnh: docker pull, docker rm -f, docker run ...

## 7. Cách build & push image lên GHCR (tóm tắt)
Nếu bạn là owner `ductoanoxo` và muốn build/push image:

1. Tạo PAT (Personal Access Token) với `write:packages` (push) và `read:packages` (pull) scope hoặc `repo` nếu dùng private repo.
2. Build image local:
```bash
# ví dụ backend
docker build -t ghcr.io/ductoanoxo/foodfast-server:my-tag ./server_app
# login
echo $GHCR_TOKEN | docker login ghcr.io -u ductoanoxo --password-stdin
# push
docker push ghcr.io/ductoanoxo/foodfast-server:my-tag
```
3. Update `SERVER_TAG` (hoặc tương ứng) trong workflow hoặc trigger workflow với giá trị mới.

## 8. Kiểm tra & Troubleshooting
- Nếu SSH connection fail: kiểm tra `PROD_SSH_KEY`, `PROD_SERVER_HOST`, `PROD_SERVER_USER` và security group (port 22 mở cho runner).
- Nếu docker login GHCR fail: kiểm tra `GHCR_TOKEN` (scope `read:packages`) và `OWNER` username.
- Nếu container không chạy: SSH vào EC2, chạy `sudo docker ps -a` và `sudo docker logs <container>` để debug.
- Nếu healthcheck fail: thử curl thủ công `curl -v http://EC2_IP:5000/health`.
- Nếu muốn rollback: thay đổi tag về tag trước đó và trigger workflow hoặc pull/run thủ công image cũ.

## 9. Gợi ý bảo mật
- Không expose port 22 ra toàn thế giới; giới hạn IP cho GitHub Actions runners (GitHub Actions public runners có dải IP động — nếu muốn an toàn hơn, cân nhắc dùng self-hosted runner hoặc VPN).
- Lưu `PROD_SSH_KEY` và `GHCR_TOKEN` trong GitHub Secrets.
- Sử dụng IAM role/instance profile cho EC2 khi cần truy xuất dịch vụ AWS thay vì đặt credentials trên instance.

---

Nếu bạn muốn, tôi có thể:
- Soạn sẵn public key / private key generation steps để bạn tạo `PROD_SSH_KEY`.
- Thêm phần hướng dẫn cụ thể cho việc build/push từng image (server, client, admin, restaurant, drone) dựa trên cấu trúc repo.
- Cập nhật workflow để hỗ trợ tag động (ví dụ nhận input `image_tag` khi `workflow_dispatch`).
